In [1]:
from imgtools.io import read_dicom_series
from readii.loaders import loadSegmentation
from readii.image_processing import *
from readii.negative_controls import *
from readii.metadata import *
import SimpleITK as sitk
from joblib import Parallel, delayed
from typing import Optional

In [2]:
randomSeed = 10
roiNames = None

In [3]:
def negControlToNIFTI(ctImage, alignedROIImage, segmentationLabel, outputDir, 
                      negControlTypeList: Optional[list] = None, update=False, randomSeed=10):
  
    if negControlTypeList is None:
        negControlTypeList = ["shuffled_full", "randomized_full", "randomized_sampled_full",
                              "shuffled_roi", "randomized_roi", "randomized_sampled_roi",
                              "shuffled_non_roi", "randomized_non_roi", "randomized_sampled_non_roi"]
        
    if not os.path.exists(outputDir):
        os.makedirs(outputDir)

    for controlType in negControlTypeList:
        print(controlType)
        outFileName = "CT_" + controlType + ".nii.gz"
        fullOutPath = os.path.join(outputDir, outFileName)

        if os.path.exists(fullOutPath) and not update:
            print(controlType, " negative control already exists.")
        else:
            negControlImage = applyNegativeControl(nc_type = controlType,
                                                baseImage = ctImage,
                                                baseROI = alignedROIImage,
                                                roiLabel = segmentationLabel,
                                                randomSeed = randomSeed)
            sitk.WriteImage(negControlImage, fullOutPath)

    return

In [5]:
imageDirPath = "/Users/katyscott/Documents/NSCLC_Radiogenomics/images/"
outputDir = "/Users/katyscott/Documents/NSCLC_Radiogenomics/images/niftis/"
if not os.path.exists(outputDir):
        print("Creating output directory:", outputDir)
        os.makedirs(outputDir)

imageFileListPath = "/Users/katyscott/Documents/NSCLC_Radiogenomics/images/.imgtools/imgtools_dicoms.csv"
pdImageInfo = matchCTtoSegmentation(imageFileListPath, segType = "SEG")

In [11]:
ctSeriesIDList = pdImageInfo["series_CT"].unique()

In [13]:
for ctSeriesID in ctSeriesIDList:
    ctSeriesInfo = pdImageInfo.loc[pdImageInfo["series_CT"] == ctSeriesID]
    patID = ctSeriesInfo.iloc[0]["patient_ID"]
    
    # Get absolute path to CT image files
    ctDirPath = os.path.join(imageDirPath, ctSeriesInfo.iloc[0]["folder_CT"])

    # Load CT by passing in specific series to find in a directory
    ctImage = read_dicom_series(path=ctDirPath, series_id=ctSeriesID)

    # Get list of segmentations to iterate over
    segSeriesIDList = ctSeriesInfo["series_seg"].unique()

    for segCount, segSeriesID in enumerate(segSeriesIDList):
            segSeriesInfo = ctSeriesInfo.loc[ctSeriesInfo["series_seg"] == segSeriesID]

            # Check that a single segmentation file is being processed
            if len(segSeriesInfo) > 1:
                # Check that if there are multiple rows that it's not due to a CT with subseries (this is fine, the whole series is loaded)
                if not segSeriesInfo.duplicated(subset=["series_CT"], keep=False).all():
                    raise RuntimeError(
                        "Some kind of duplication of segmentation and CT matches not being caught. Check seg_and_ct_dicom_list in radiogenomic_output."
                    )

            # Get absolute path to segmentation image file
            segFilePath = os.path.join(
                imageDirPath, segSeriesInfo.iloc[0]["file_path_seg"]
            )
            # Get dictionary of ROI sitk Images for this segmentation file
            segImages = loadSegmentation(
                segFilePath,
                modality=segSeriesInfo.iloc[0]["modality_seg"],
                baseImageDirPath=ctDirPath,
                roiNames=roiNames,
            )

            # Check that this series has ROIs to extract from (dictionary isn't empty)
            if not segImages:
                print(
                    "CT ",
                    ctSeriesID,
                    "and segmentation ",
                    segSeriesID,
                    " has no ROIs or no ROIs with the label ",
                    roiNames,
                    ". Moving to next segmentation.",
                )

            else:
                # Loop over each ROI contained in the segmentation to perform radiomic feature extraction
                for roiCount, roiImageName in enumerate(segImages):
                    # Get sitk Image object for this ROI
                    roiImage = segImages[roiImageName]

                    # Exception catch for if the segmentation dimensions do not match that original image
                    try:
                        # Check if segmentation just has an extra axis with a size of 1 and remove it
                        if roiImage.GetDimension() > 3 and roiImage.GetSize()[3] == 1:
                            roiImage = flattenImage(roiImage)

                        # Check that image and segmentation mask have the same dimensions
                        if ctImage.GetSize() != roiImage.GetSize():
                            # Checking if number of segmentation slices is less than CT
                            if ctImage.GetSize()[2] > roiImage.GetSize()[2]:
                                print(
                                    "Slice number mismatch between CT and segmentation for",
                                    patID,
                                    ". Padding segmentation to match.",
                                )
                                roiImage = padSegToMatchCT(
                                    ctDirPath, segFilePath, ctImage, roiImage
                                )
                            else:
                                raise RuntimeError(
                                    "CT and ROI dimensions do not match."
                                )

                    # Catching CT and segmentation size mismatch error
                    except RuntimeError as e:
                        print(str(e))

                    alignedROIImage = alignImages(ctImage, roiImage)
                    segmentationLabel = getROIVoxelLabel(alignedROIImage)

                    completeOutputPath = os.path.join(outputDir, patID) 
                    
                    negControlToNIFTI(ctImage, alignedROIImage, segmentationLabel, completeOutputPath, randomSeed=randomSeed)
    
                    
## SHUFFLED ROI TAKES WAY TOO LONG WITH THE IMAGE AS IT IS, NEEDS TO BE CROPPED OR OPTIMIZED

shuffled_full
shuffled_full  negative control already exists.
randomized_full
randomized_full  negative control already exists.
randomized_sampled_full
randomized_sampled_full  negative control already exists.
shuffled_roi
randomized_roi
randomized_sampled_roi
shuffled_non_roi
randomized_non_roi
randomized_sampled_non_roi


KeyboardInterrupt: 

In [22]:
os.path.join(outputDir, patID)

'/Users/katyscott/Documents/NSCLC_Radiogenomics/images/niftis/R01-001'

In [12]:
import datetime
import time

# datetime.timedelta(seconds=elapsed)

NameError: name 'elapsed' is not defined

In [15]:
start = time.time()
print("Testing")
end = time.time()

print(datetime.timedelta(time.time() - start))

Testing
0:00:12.606812


In [1]:
import yaml

config = yaml.safe_load(open("../config.yaml"))

In [1]:
import pandas as pd
roiNameList = pd.read_csv("/Users/katyscott/Documents/TCIA_TCGA_Datasets/Head-Neck-PET-CT/roi_names_TCIA_Head-Neck-PET-CT.csv")

In [20]:
roiNameList[roiNameList.roi_names.str.fullmatch('GTVp|GTV[\s-][pP]rimary', case=False)]

,roi_names,count
746,GTV Primary,2
763,GTV primary,7
780,GTV-Primary,1
814,GTVp,403
2004,gtv primary,3


In [9]:
roiNameList[roiNameList.roi_names.str.fullmatch('GTV T et N\+|GTV 2 T et N\+|GTV  66 Gy', case=False)]

,roi_names,count
661,GTV 66 Gy,2
663,GTV 2 T et N+,2
687,GTV T et N+,2


In [10]:
import os 
import re

patID = "RADCURE-0065"
radOutputDir = "/Users/katyscott/Documents/RADCURE/nc_niftis"
pattern = patID + "*"

for filename in os.listdir(radOutputDir):
    if re.search(pattern, filename):
        print(os.path.join(radOutputDir, filename))


/Users/katyscott/Documents/RADCURE/nc_niftis/RADCURE-0065_2


In [6]:
patID + "*"

'RADCURE-0020*'

In [13]:
import pandas as pd
import os
from imgtools.io import read_dicom_series
from readii.loaders import loadSegmentation
from readii.image_processing import alignImages
from readii.negative_controls import applyNegativeControl
from joblib import Parallel, delayed
import SimpleITK as sitk

# load edge file
edgeGraphPath = "/Users/katyscott/Documents/RADCURE/.imgtools/imgtools_RADCURE_edges.csv"
imageDirPath = "/Users/katyscott/Documents/RADCURE/"
roiNames = ["GTVp*"]
randomSeed = 10

dfEdgeGraph = pd.read_csv(edgeGraphPath)

dfCTWithRTSTRUCTS = dfEdgeGraph.loc[dfEdgeGraph['edge_type'] == 2]
ctSeriesIDList = dfCTWithRTSTRUCTS["series_x"].unique()

outputDir = "/Users/katyscott/Documents/HNC Project/scripts/negative-control-to-nifti/nc_niftis/"

controlType = ["shuffled", "randomized", "randomized_sampled"]
controlRegion = ["full", "roi", "non_roi"]

if not os.path.exists(outputDir):
    print("Creating output directory:", outputDir)
    os.makedirs(outputDir)
    
def ctProcessing(ctSeriesID):
    ctSeriesInfo = dfCTWithRTSTRUCTS.loc[dfCTWithRTSTRUCTS["series_x"] == ctSeriesID]
    patID = ctSeriesInfo.iloc[0]["patient_ID_x"]
    print("Processing ", patID, "...")

    patOutDir = os.path.join(outputDir, patID)
    if not os.path.exists(patOutDir):
        os.makedirs(patOutDir)

    ctOutDir = os.path.join(patOutDir, "CT")
    if not os.path.exists(ctOutDir):
        os.makedirs(ctOutDir)
    
    rtOutDir = os.path.join(patOutDir, "RTSTRUCT_CT")
    if not os.path.exists(rtOutDir):
        os.makedirs(rtOutDir)
    
    # load CT from DICOM
    ctDirPath = os.path.join(imageDirPath, ctSeriesInfo.iloc[0]["folder_x"])
    ctImage = read_dicom_series(path=ctDirPath, series_id=ctSeriesID)

    # Load RTSTRUCT from DICOM using regex
    segFilePath = os.path.join(imageDirPath, ctSeriesInfo.iloc[0]["file_path_y"])
    segImages = loadSegmentation(segFilePath,
                                modality=ctSeriesInfo.iloc[0]["modality_y"],
                                baseImageDirPath=ctDirPath,
                                roiNames=roiNames)
    # Get the first ROI mask
    roiLabel = list(segImages.keys())[0]
    roiMask = segImages[roiLabel]

    # Align the ROI mask to the CT image
    roiMask = alignImages(ctImage, roiMask)

    # Save CT to nifti - CT.nii.gz
    sitk.WriteImage(ctImage, os.path.join(ctOutDir, "CT.nii.gz"))
    # Save RTSTRUCT to nifti - GTVp.nii.gz
    sitk.WriteImage(roiMask, os.path.join(rtOutDir, "GTVp.nii.gz"))

    # Save each negative control to nifti - CT_negative_control_name.nii.gz
    for type in controlType:
        for region in controlRegion:
            print(type, region)
            outFileName = "CT_" + type + "_" + region + ".nii.gz"
            fullOutPath = os.path.join(ctOutDir, outFileName)
        
            ncCTImage = applyNegativeControl(baseImage=ctImage,
                                negativeControlType = type,
                                negativeControlRegion= region,
                                roiMask = roiMask,
                                randomSeed = randomSeed)
            
            sitk.WriteImage(ncCTImage, fullOutPath)

    return

Parallel(n_jobs=-1, require="sharedmem")(delayed(ctProcessing)(ctSeriesID) for ctSeriesID in ctSeriesIDList)
print("Pipeline complete")

    
# filter by edge type 2
# parallelize from here on down
# for each ctseriesid
    # make patient dir
    # make CT dir
    # make RTSTRUCT dir
    # load CT from DICOM 
    # save CT to nifti - CT.nii.gz
    # load RTSTRUCT from DICOM using regex 
    # save RTSTRUCT to nifti - regex.nii.gz
    # for each negative control
        # make the negative control, not cropped
        # save to CT dir - CT_negative_control_name.nii.gz 

Processing Processing  RADCURE-0020 ...
 RADCURE-0065 ...
Processing  RADCURE-0099 ...
Processing  RADCURE-0112 ...
labels: {'GTVp': 0}
labels: {'GTVp': 0}
labels: {'GTVp': 0}
shuffled full
labels: {'GTVp': 0}
shuffled full
shuffled full
shuffled roi
shuffled full
shuffled roi
shuffled non_roi
shuffled non_roi
randomized full
shuffled roi
shuffled roi
randomized full
shuffled non_roi
shuffled non_roi
randomized roi
randomized roi
randomized non_roi
randomized non_roi
randomized_sampled full
randomized_sampled full
randomized full
randomized full
randomized_sampled roi
randomized_sampled roi
randomized_sampled non_roi
randomized_sampled non_roi
randomized roi
randomized roi
randomized non_roi
randomized non_roi
randomized_sampled full
randomized_sampled full
randomized_sampled roi
randomized_sampled roi
randomized_sampled non_roi
randomized_sampled non_roi
Pipeline complete
